### Inspecting Data in Spark

In this notebook we demonstrate how to examine the data and metadata (schema) in Spark DataFrames. This notebook is based on material supplied by Cloudera under their Cloudera Academic Partner program and the *Spark: The Definitive Guide* book by Bill Chambers and Matei Zaharia. You can find out more about Spark here: [https://spark.apache.org/](https://spark.apache.org/ "Apache Spark"). 

Topics
In this module we will take our first look at a Spark DataFrame.
- Examining the schema
- Viewing some data
- Computing the number of rows (observations)
- Computing summary statistics
- Inspecting a column (variable)
-- Inspecting a key variable
-- Inspecting a categorical variable
-- Inspecting a continuous numerical variable
-- Inspecting a datetime variable

In [0]:
# Load the rider data from our location for data on S3 into a Spark DataFrame
riders = spark.read.csv("/mnt/cis442f-data/duocar/raw/riders/", header=True, inferSchema=True)


### Examining the schema

Use the `printSchema` method of the DataFrame class to examine the schema in a tree format. Follow [this link](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame) to see the full set of Spark DataFrame methods. You can tell that it is a method as it follows the syntactical format of `instance_of_class.method_name()` e.g. `riders.printSchema()`

In [0]:
riders.printSchema()

root
-- id: long (nullable = true)
-- birth_date: string (nullable = true)
-- start_date: string (nullable = true)
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- sex: string (nullable = true)
-- ethnicity: string (nullable = true)
-- student: integer (nullable = true)
-- home_block: long (nullable = true)
-- home_lat: double (nullable = true)
-- home_lon: double (nullable = true)
-- work_lat: double (nullable = true)
-- work_lon: double (nullable = true)

Use the `dtypes`, `columns`, and `schema` attributes to view the schema in other ways

**Note:** 
- The `schema` attribute provides a programmatic version of the schema. 
- You can tell that it is an attribute as it follows the syntactical format of `instance_of_class.attribute_name` e.g. `riders.schema`. There are no parentheses `()` after an attribute name. 
- `dtypes` and `columns` return python lists

In [0]:
riders.dtypes 

Out[3]: [('id', 'bigint'),
 ('birth_date', 'string'),
 ('start_date', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('sex', 'string'),
 ('ethnicity', 'string'),
 ('student', 'int'),
 ('home_block', 'bigint'),
 ('home_lat', 'double'),
 ('home_lon', 'double'),
 ('work_lat', 'double'),
 ('work_lon', 'double')]

In [0]:
riders.columns

Out[4]: ['id',
 'birth_date',
 'start_date',
 'first_name',
 'last_name',
 'sex',
 'ethnicity',
 'student',
 'home_block',
 'home_lat',
 'home_lon',
 'work_lat',
 'work_lon']

In [0]:
riders.schema

Out[5]: StructType(List(StructField(id,LongType,true),StructField(birth_date,StringType,true),StructField(start_date,StringType,true),StructField(first_name,StringType,true),StructField(last_name,StringType,true),StructField(sex,StringType,true),StructField(ethnicity,StringType,true),StructField(student,IntegerType,true),StructField(home_block,LongType,true),StructField(home_lat,DoubleType,true),StructField(home_lon,DoubleType,true),StructField(work_lat,DoubleType,true),StructField(work_lon,DoubleType,true)))

####Viewing some data and counting the number of rows (observations)

- Use the `show` method to get a SQL-like display of the data
- Use the `head` or `take` method to get a display of the `Row` objects that is sometimes easier to read
- Use the `count` method to compute the number of rows

**Notes:** 
- `id` and `home_block` are long integers rather than strings.
- `birth_date` and `start_date` are Timestamps rather than Dates.
- `ethnicity`, `work_lat`, and `work_lon` appear to have null (missing) values.
- `student` is a boolean variable inefficiently encoded as an integer.

In [0]:
# Use the `show` method to get a SQL-like display of the data
riders.show(3)

+------------+----------+----------+----------+---------+------+---------+-------+---------------+---------+----------+---------+----------+
 id|birth_date|start_date|first_name|last_name| sex|ethnicity|student| home_block| home_lat| home_lon| work_lat| work_lon|
+------------+----------+----------+----------+---------+------+---------+-------+---------------+---------+----------+---------+----------+
220200000001|1962-03-18|2017-01-01| Natalie| Prosser|female| White| 0|380170405002188|46.816399|-96.874038|46.831427|-96.827786|
220200000002|1981-10-06|2017-01-01| Nicholas| Murray| male| White| 0|380170405002360|46.808599| -96.85689| null| null|
220200000003|1994-12-05|2017-01-01| Samuel| Zinanti| male| null| 0|380170103071039|46.821603|-96.806238| null| null|
+------------+----------+----------+----------+---------+------+---------+-------+---------------+---------+----------+---------+----------+
only showing top 3 rows

In [0]:
# You can choose to show a subset of columns for neater printing
# Copying and pasting output from the columns attribute saves you from having to type all the column names 
riders.select(['id', 'first_name',  'last_name', 'sex',  'ethnicity',  'student',  'home_block']).show(3)


+------------+----------+---------+------+---------+-------+---------------+
 id|first_name|last_name| sex|ethnicity|student| home_block|
+------------+----------+---------+------+---------+-------+---------------+
220200000001| Natalie| Prosser|female| White| 0|380170405002188|
220200000002| Nicholas| Murray| male| White| 0|380170405002360|
220200000003| Samuel| Zinanti| male| null| 0|380170103071039|
+------------+----------+---------+------+---------+-------+---------------+
only showing top 3 rows

In [0]:
# Use the `head` or `take` method to get a display of the `Row` objects. 
# Not much difference between these methods but must specify number of lines with `take`. The default for `head()` is 1 
# Perhaps `head` was included to mirror pandas
riders.head()

Out[8]: Row(id=220200000001, birth_date='1962-03-18', start_date='2017-01-01', first_name='Natalie', last_name='Prosser', sex='female', ethnicity='White', student=0, home_block=380170405002188, home_lat=46.816399, home_lon=-96.874038, work_lat=46.831427, work_lon=-96.827786)

In [0]:
# Use the `head` or `take` method to get a display of the `Row` objects 
riders.take(2)

Out[9]: [Row(id=220200000001, birth_date='1962-03-18', start_date='2017-01-01', first_name='Natalie', last_name='Prosser', sex='female', ethnicity='White', student=0, home_block=380170405002188, home_lat=46.816399, home_lon=-96.874038, work_lat=46.831427, work_lon=-96.827786),
 Row(id=220200000002, birth_date='1981-10-06', start_date='2017-01-01', first_name='Nicholas', last_name='Murray', sex='male', ethnicity='White', student=0, home_block=380170405002360, home_lat=46.808599, home_lon=-96.85689, work_lat=None, work_lon=None)]

In [0]:
# Use the `count` method to compute the number of rows
riders.count()

Out[10]: 1723

#### Computing summary statistics

Use the `describe` method to compute summary statistics for numeric and string columns

**Notes:** 
- The count in the `describe` method is a count of non-missing values.
- `sex` and `ethnicity` seem to have some missing values.

In [0]:
# Use the `describe` method to compute summary statistics
# but asking for all columns can be difficult to read
riders.describe().show() 


+-------+--------------------+----------+----------+----------+---------+------+---------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+
summary| id|birth_date|start_date|first_name|last_name| sex|ethnicity| student| home_block| home_lat| home_lon| work_lat| work_lon|
+-------+--------------------+----------+----------+----------+---------+------+---------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+
 count| 1723| 1723| 1723| 1723| 1723| 1646| 1551| 1723| 1723| 1723| 1723| 866| 866|
 mean|2.202000011309907...| null| null| null| null| null| null|0.12362159024956472|3.508294844708125E14| 46.861779514799885| -96.80689443760858| 46.85815218591215| -96.81390317667442|
 stddev| 657.2877906803506| null| null| null| null| null| null|0.32924490501424786|4.863160498672944...|0.061215721903146565|0.14933258235905558|0.03517133668654094|0.08811640407110832|
 min| 220200000001|1928-08-01|2017-01-01| Aaron| Aadson|female| Asian| 0| 270270201001010| 46.633395| -97.671608| 46.645552| -97.491754|
 max| 220200002286|1999-03-30|2017-04-22| Zoiee| Zufelt| male| White| 1| 380170406003040| 47.237408| -96.178997| 47.079167| -96.260833|
+-------+--------------------+----------+----------+----------+---------+------+---------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+

#### Inspecting a column (variable)

We often want to inspect one or a few variables at a time.
 

#### Inspect a key variable ###

The variable `id` should be a unique value.  Let us confirm this.  Use the `select` method to select the `id` column.
**Note** 
- We can express our requirements in either **DataFrame** or **SQL** syntax 
- We create a view for the SQL approach using `riders.createOrReplaceTempView("riders")`

In [0]:
riders.select("id").describe().show()

+-------+--------------------+
summary| id|
+-------+--------------------+
 count| 1723|
 mean|2.202000011309907...|
 stddev| 657.2877906803506|
 min| 220200000001|
 max| 220200002286|
+-------+--------------------+

In [0]:
# Alternatively, pass the column name to the `describe` method:
riders.describe("id").show()

+-------+--------------------+
summary| id|
+-------+--------------------+
 count| 1723|
 mean|2.202000011309907...|
 stddev| 657.2877906803506|
 min| 220200000001|
 max| 220200002286|
+-------+--------------------+

In [0]:
# Asking for a few columns at a time can make output easier to read
riders.describe(['id', 'birth_date', 'start_date', 'first_name', 'last_name','sex', 'ethnicity']).show() 
riders.describe(['student', 'home_block','home_lat']).show() 
riders.describe('home_lon', 'work_lat', 'work_lon').show() 

+-------+--------------------+----------+----------+----------+---------+------+---------+
summary| id|birth_date|start_date|first_name|last_name| sex|ethnicity|
+-------+--------------------+----------+----------+----------+---------+------+---------+
 count| 1723| 1723| 1723| 1723| 1723| 1646| 1551|
 mean|2.202000011309907...| null| null| null| null| null| null|
 stddev| 657.2877906803506| null| null| null| null| null| null|
 min| 220200000001|1928-08-01|2017-01-01| Aaron| Aadson|female| Asian|
 max| 220200002286|1999-03-30|2017-04-22| Zoiee| Zufelt| male| White|
+-------+--------------------+----------+----------+----------+---------+------+---------+

+-------+-------------------+--------------------+--------------------+
summary| student| home_block| home_lat|
+-------+-------------------+--------------------+--------------------+
 count| 1723| 1723| 1723|
 mean|0.12362159024956472|3.508294844708125E14| 46.861779514799885|
 stddev|0.32924490501424786|4.863160498672944...|0.061215721903146565|
 min| 0| 270270201001010| 46.633395|
 max| 1| 380170406003040| 47.237408|
+-------+-------------------+--------------------+--------------------+

+-------+-------------------+-------------------+-------------------+
summary| home_lon| work_lat| work_lon|
+-------+-------------------+-------------------+-------------------+
 count| 1723| 866| 866|
 mean| -96.80689443760858| 46.85815218591215| -96.81390317667442|
 stddev|0.14933258235905558|0.03517133668654094|0.08811640407110832|
 min| -97.671608| 46.645552| -97.491754|
 max| -96.178997| 47.079167| -96.260833|
+-------+-------------------+-------------------+-------------------+

In [0]:
# Use the `countDistinct` function to determine the number of distinct values
from pyspark.sql.functions import count, countDistinct

riders.select(count("id"), countDistinct("id")).show()

# Note - This can take quite a bit of time on large DataFrames.

+---------+------------------+
count(id)|count(DISTINCT id)|
+---------+------------------+
 1723| 1723|
+---------+------------------+

In [0]:
# You can use functional style, as shown above, or SQL style for examination of
# DataFrames.  SQL style requires one preliminary step:

riders.createOrReplaceTempView("riders")

spark.sql("select count(id), count(distinct id) from riders").show()

+---------+------------------+
count(id)|count(DISTINCT id)|
+---------+------------------+
 1723| 1723|
+---------+------------------+

#### Inspect a categorical variable

The variable `sex` is a categorical variable.  Let us examine it more carefully.
Note that 
- `countDistinct` function does not count `NULL` as a distinct value but that the `.distinct()` method does
- We can use DataFrame or SQL format to carry-out our examination
- We can execute SQL using `spark.sql` syntax and the previously defined 'view'
- We can also write an SQL statement directly in a notebook paragraph with the `%sql` prefix (also works in Zeppelin)

In [0]:
riders.select(count("*"), count("sex"), countDistinct("sex")).show()

+--------+----------+-------------------+
count(1)|count(sex)|count(DISTINCT sex)|
+--------+----------+-------------------+
 1723| 1646| 2|
+--------+----------+-------------------+

In [0]:
# Use the `select` method to select a subset of columns:
riders.select("sex").show(5)

+------+
 sex|
+------+
female|
 male|
 male|
female|
 male|
+------+
only showing top 5 rows

In [0]:
# Use the `distinct` method to determine the unique values of `sex`
riders.select("sex").distinct().show() 
riders.select("sex").distinct().count()

+------+
 sex|
+------+
 null|
female|
 male|
+------+

Out[19]: 3

In [0]:
# An alternative approach is to use an aggregation
riders.groupby("sex").count().show()

# The same query in SQL style (note that "group by 1" is reffering to the first column in the select statement)
spark.sql("select sex, count(*) from riders group by 1").show()
  
# **Note:** `sex` contains null (missing) values that we may have to deal with

+------+-----+
 sex|count|
+------+-----+
 null| 77|
female| 694|
 male| 952|
+------+-----+

+------+--------+
 sex|count(1)|
+------+--------+
 null| 77|
female| 694|
 male| 952|
+------+--------+

You can use the `%sql` magic command to use SQL directly on the view created above. 
Databricks notebooks (and Zeppelin notebooks) include some basic visualization abilities 
on the output of queries

See https://docs.databricks.com/user-guide/visualizations/index.html#visualizations-in-python for more on visualizations in Databricks notebooks

In [0]:
%sql 
select sex, count(*) AS `Number of Riders`
from riders 
group by sex


sex,Number of Riders
null,77
female,694
male,952


In [0]:
# Databricks offers a direct way of visualizing data in Spark DataFrames
display(riders.groupBy("sex").count())

sex,count
null,77
female,694
male,952


#### Inspecting a numerical variable

**Notes:**
- No missing values
- No extreme values (tight distribution about [Fargo, North Dakota](https://en.wikipedia.org/wiki/Fargo%2C_North_Dakota))

In [0]:
riders.select("home_lat", "home_lon").show(5)
riders.select("home_lat", "home_lon").describe().show(5)


+---------+----------+
 home_lat| home_lon|
+---------+----------+
46.816399|-96.874038|
46.808599| -96.85689|
46.821603|-96.806238|
46.873851|-96.906226|
46.857253|-96.831356|
+---------+----------+
only showing top 5 rows

+-------+--------------------+-------------------+
summary| home_lat| home_lon|
+-------+--------------------+-------------------+
 count| 1723| 1723|
 mean| 46.861779514799885| -96.80689443760858|
 stddev|0.061215721903146565|0.14933258235905558|
 min| 46.633395| -97.671608|
 max| 47.237408| -96.178997|
+-------+--------------------+-------------------+

In [0]:
# Use the `approxQuantile` to get customized (approximate) quantiles:
riders.approxQuantile(["home_lat","home_lon"],  \
	probabilities=[0.0, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 1.0], \
	relativeError=0.1)

# **Note:** This method returns a Python list. 

Out[23]: [[46.633395,
 46.633395,
 46.633395,
 46.840907,
 46.856614,
 46.876205,
 47.237408,
 47.237408,
 47.237408],
 [-97.671608,
 -97.671608,
 -97.671608,
 -96.844913,
 -96.815119,
 -96.78926,
 -96.178997,
 -96.178997,
 -96.178997]]

#### Inspecting a datetime variable

Note that the original data was in Date format, but Spark read the data in
Timestamp format.  We will probably want to fix this.

In [0]:
# Let us inspect `birth_date` and `start_date`, which are both Timestamp
# variables:

dates = riders.select("birth_date", "start_date")
dates.show(5, truncate=False)
dates.head(5)

+----------+----------+
birth_date|start_date|
+----------+----------+
1962-03-18|2017-01-01|
1981-10-06|2017-01-01|
1994-12-05|2017-01-01|
1970-05-31|2017-01-01|
1962-12-12|2017-01-01|
+----------+----------+
only showing top 5 rows

Out[24]: [Row(birth_date='1962-03-18', start_date='2017-01-01'),
 Row(birth_date='1981-10-06', start_date='2017-01-01'),
 Row(birth_date='1994-12-05', start_date='2017-01-01'),
 Row(birth_date='1970-05-31', start_date='2017-01-01'),
 Row(birth_date='1962-12-12', start_date='2017-01-01')]

In [0]:
# Note that the `describe` method does not work with Date or Timestamp
# variables:

dates.describe().show(5)

+-------+----------+----------+
summary|birth_date|start_date|
+-------+----------+----------+
 count| 1723| 1723|
 mean| null| null|
 stddev| null| null|
 min|1928-08-01|2017-01-01|
 max|1999-03-30|2017-04-22|
+-------+----------+----------+

###Hands On

![Hands-on](https://cis442f-open-data.s3.amazonaws.com/pictures/hands.png "Hands-on")


#### Exercises

(1) Read the raw driver data from HDFS into a Spark DataFrame.

(2) Inspect the driver DataFrame.  Are the data types for each column appropriate?

(3) Inspect the columns of the driver DataFrame.  Are there any issues with the data?



**References you might need to learn about Spark and Databricks**

[Spark SQL, DataFrames, and Datasets Guide](http://spark.apache.org/docs/latest/sql-programming-guide.html)

[DataFrame class](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#dataframe-apis)

[Spark Types](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#data-types)

[Data visualizations in Databricks](https://docs.databricks.com/user-guide/visualizations/index.html#visualizations-in-python)